In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import tensorflow
from tensorflow.keras.models import Model, Sequential
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.layers import Flatten, Dense, Dropout, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose, Conv2D, Input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img

In [0]:
def get_real_images(directory):
  images = []
  for image in os.listdir(directory):
    image = load_img(os.path.join(directory,image), target_size=(28,28))
    image = img_to_array(image)
    images.append(image)
  return (np.array(images)/127.5)-1

In [0]:
def Generator():
  model = Sequential()
  
  model.add(Dense(7*7*256, use_bias=False, input_shape = (100,)))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  
  model.add(Reshape((7,7,256)))
  assert model.output_shape == (None, 7, 7, 256)
  
  model.add(Conv2DTranspose(128, (5,5), strides = (1, 1), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  assert model.output_shape == (None, 7, 7, 128)
  
  model.add(Conv2DTranspose(64, (5,5), strides = (2, 2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  assert model.output_shape == (None, 14, 14, 64)
  
  model.add(Conv2DTranspose(1, (5,5), strides = (2, 2), padding='same', activation='tanh'))
  assert model.output_shape == (None, 28, 28, 1)
  
  return model

In [0]:
def Discriminator():

  model = Sequential()

  model.add(Conv2D(32, (5,5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Conv2D(64, (5,5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Conv2D(128, (5,5), strides=(2, 2), padding='same'))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))

  return model



In [0]:
def GAN(generator, discriminator):
  noise = Input(shape=(100,))
  generator_output = generator(noise)
  discriminator_output = discriminator(generator_output)
  gan = Model(noise, discriminator_output)
  return gan

In [0]:
generator = Generator()

discriminator = Discriminator()
descriminator.compile(optimizer=Optimizer.Adam(lr=0.0003), loss='binary_crossentropy', metrics=['binary_accuracy'])

gan = GAN(generator, discriminator)
gan.compile(optimizer=Optimizer.Adam(lr=0.0003), loss='binary_crossentropy')

real = np.ones((32,1))
fake = np.zeros((32,1))

batch_size = 32

images = get_real_images('/content/100k/')

for e in range(epochs):
  
  for i in range(len(images)//batch_size):
    
    Real_batch = images[i*batch_size:(i+1)*batch_size]
    
    noise = np.random.normal(loc=0, scale=1, size=(32, 100))
    
    real_loss = discriminator.train_on_batch(x = Real_batch, y = real)

    Fake_batch = generator.predict_on_batch(noise)

    fake_loss = discriminator.train_on_batch(x = Fake_batch, y = fake)

    discriminator_loss = np.add(descri_loss_real, descri_loss_fake)/2

    gan_loss = gan.train_on_batch(noise, real)

    print(e, discriminator_loss, gan_loss)